<a href="https://colab.research.google.com/github/rpmullig/AlgorithmsMPCS/blob/master/CIFAR_100_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

# Confirm that we're using Python 3
assert sys.version_info.major is 3, 'Oops, not running Python 3. Use Runtime > Change runtime type'

print("Installing dependencies for Colab environment")
!pip install -Uq grpcio==1.26.0

import tensorflow as tf
from tensorflow import keras

import tensorflow_datasets as tfds

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

print('TensorFlow version: {}'.format(tf.__version__))


# turn on TPU https://www.tensorflow.org/guide/tpu
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

Installing dependencies for Colab environment
TensorFlow version: 2.2.0
INFO:tensorflow:Initializing the TPU system: grpc://10.74.150.90:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.74.150.90:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [0]:
def get_dataset(batch_size=200):
  datasets, info = tfds.load(name='cifar100', with_info=True, as_supervised=True,
                             try_gcs=True)
  cifar_train, cifar_test = datasets['train'], datasets['test']

  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0

    return image, label

  train_dataset = cifar_train.map(scale).shuffle(10000).batch(batch_size)
  test_dataset = cifar_test.map(scale).batch(batch_size)

  return train_dataset, test_dataset

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

with strategy.scope():
  model = Sequential()

  model.add(Conv2D(128, (3, 3), padding='same', input_shape=(32,32,3)))
  model.add(Activation('elu'))
  model.add(Conv2D(128, (3, 3)))
  model.add(Activation('elu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  #model.add(Dropout(0.25))

  model.add(Conv2D(256, (3, 3), padding='same'))
  model.add(Activation('elu'))
  model.add(Conv2D(256, (3, 3)))
  model.add(Activation('elu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(512, (3, 3), padding='same'))
  model.add(Activation('elu'))
  model.add(Conv2D(512, (3, 3)))
  model.add(Activation('elu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))


  model.add(Flatten())
  model.add(Dense(1024))
  model.add(Activation('elu'))
  model.add(Dropout(0.5))
  model.add(Dense(100))
  model.add(Activation('softmax'))

  model.compile(optimizer='adam',
                loss=(tf.reduce_sum(loss_obj(labels, predictions)) * (1. / global_batch_size)),
                metrics=['sparse_categorical_accuracy'])

# call function
train_dataset, test_dataset = get_dataset()

# fit the model
model.fit(train_dataset,
          epochs=10,
          validation_data=test_dataset)


test_loss, test_acc = model.evaluate(test_images, test_labels,)
print('\n\nTest accuracy: {}'.format(test_acc))

NameError: ignored

In [29]:
f

with strategy.scope():
  model = Sequential()

  model.add(Conv2D(128, (3, 3), padding='same', input_shape=(32,32,3)))
  model.add(Activation('elu'))
  model.add(Conv2D(128, (3, 3)))
  model.add(Activation('elu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  #model.add(Dropout(0.25))

  model.add(Conv2D(256, (3, 3), padding='same'))
  model.add(Activation('elu'))
  model.add(Conv2D(256, (3, 3)))
  model.add(Activation('elu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(512, (3, 3), padding='same'))
  model.add(Activation('elu'))
  model.add(Conv2D(512, (3, 3)))
  model.add(Activation('elu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))


  model.add(Flatten())
  model.add(Dense(1024))
  model.add(Activation('elu'))
  model.add(Dropout(0.5))
  model.add(Dense(100))
  model.add(Activation('softmax'))

model.summary()

testing = False
epochs = 200

opt = tf.keras.optimizers.RMSprop(
    learning_rate=0.001, rho=0.9, momentum=0.0, epsilon=1e-07, centered=False,
    name='RMSprop', 
)

model.compile(optimizer=opt, 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],)

model.fit(train_images, train_labels, epochs=epochs)

test_loss, test_acc = model.evaluate(test_images, test_labels,)
print('\nTest accuracy: {}'.format(test_acc))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 32, 32, 128)       3584      
_________________________________________________________________
activation_25 (Activation)   (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 30, 30, 128)       147584    
_________________________________________________________________
activation_26 (Activation)   (None, 30, 30, 128)       0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 15, 15, 128)       0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 15, 15, 256)       295168    
_________________________________________________________________
activation_27 (Activation)   (None, 15, 15, 256)      

AttributeError: ignored

In [22]:
import numpy as np 

test_data = np.load("cifar100_test.npy")

test_data = test_data / 255.0


test_data = test_data.reshape(test_data.shape[0], 32, 32, 3)


print('test_data.shape: {}, of {}'.format(test_data.shape, test_data.dtype))

predictions = model.predict(test_data)

output = np.array(np.argmax(predictions[0]), dtype=np.float64)

for i in range(1, predictions.shape[0]):
  output = np.append(output, np.argmax(predictions[i]))


import pandas as pd
from google.colab import files

df = pd.DataFrame(output, columns=["Category"])
df.index.name = "Id"
df.to_csv('predictions_1.csv')
files.download('predictions_1.csv')

test_data.shape: (10000, 32, 32, 3), of float64
